# SQLite

## SQL Lite Command Line

- .headers on  which switches column headers on
- .mode allows us to select from a few different display modes. We'll use .mode column to allow for easier to read outputs. 
- .help - Displays help text showing all dot commands and their function.
- .tables - Displays a list of all tables and views in the current database.
- .shell [command] - Run a command like ls or clear in the system shell.
- .quit - Quits the SQLite shell.
- .schema [table_name]



## Get info on a table

In [ ]:
PRAGMA TABLE_INFO(recent_grads)

## Use the ‘HAVING’ clause

In [ ]:
SELECT Major_category, ROUND(AVG(College_jobs)/AVG(Total),3) as share_degree_jobs
FROM recent_grads
GROUP BY Major_category
HAVING share_degree_jobs < 0.3

## Subqueries

The query that replaces the placeholder subquery needs to be a full query (contain SELECT and FROM clauses, etc), that works even if it's run separately. In addition, the inner query should only return a table with a single row and column because of where it fits in the outer query (... WHERE > ?).

To dynamically calculate the number of total rows:
And be able to use it in another SQL statement, we can use a subquery in the SELECT clause:

In [ ]:
## 2. Subqueries ##

SELECT Major, Unemployment_rate
FROM recent_grads
WHERE Unemployment_rate < (SELECT AVG(Unemployment_rate) FROM recent_grads)
ORDER BY Unemployment_rate

## 3. Subquery In SELECT ##

SELECT CAST(COUNT(*) as Float)/CAST((SELECT COUNT(*) FROM recent_grads) as Float) as proportion_abv_avg
FROM recent_grads
WHERE ShareWomen > (SELECT AVG(ShareWomen) FROM recent_grads)

## 4. Returning Multiple Results In Subqueries ##

SELECT Major, Major_category
FROM recent_grads
WHERE Major_category IN (SELECT Major_category FROM recent_grads GROUP BY major_category ORDER BY SUM(Total) DESC LIMIT 5)


## 5. Building Complex Subqueries ##

SELECT AVG(CAST(Sample_size AS Float)/CAST(Total AS Float)) avg_ratio
FROM recent_grads

## 6. Practice Integrating A Subquery With The Outer Query ##

SELECT Major, Major_category, CAST(Sample_size AS Float)/CAST(Total AS Float) ratio
FROM recent_grads
WHERE ratio > (SELECT AVG(CAST(Sample_size AS Float)/CAST(Total AS Float)) avg_ratio
FROM recent_grads)


## Comparing to NULL

When making a comparison to null in SQL, we use the IS keyword, rather than the = sign. If we want to select rows where a column is null we can write:

In [ ]:
WHERE column_name IS NULL
WHERE column_name IS NOT NULL

## Pattern Matching Using Like ##

In [ ]:
SELECT first_name, last_name, phone
FROM customer
WHERE first_name LIKE "%Belle%"

## Create Tables

In [ ]:
CREATE TABLE [table_name] (
        [column1_name] [column1_type],
       [column2_name] [column2_type],
       [column3_name] [column3_type],
        [...]
);

## Primary & Foreign Keys 
By default SQLite doesn't force foreign key constraints, In SQLite, if you have an integer primary key and don't specify a value for this column when inserting rows, SQLite will autoincrement this column for you.

In [ ]:
CREATE TABLE purchase (
    purchase_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    purchase_date TEXT,
    total NUMERIC,
    FOREIGN KEY (user_id) REFERENCES user(user_id)
);

## Compound primary key

In [ ]:
CREATE TABLE [table_name] (
    [column_one_name] [column_one_type],
    [column_two_name] [column_two_type],
    [column_three_name] [column_three_type],
    [column_four_name] [column_four_type],
    PRIMARY KEY (column_one_name, column_two_name)
);

## Add Column

In [ ]:
ALTER TABLE [table_name]
ADD COLUMN [column_name] [column_type];

## Data Types

In [ ]:
TEXT
INTEGER
REAL
NUMERIC
BLOB

## Add Rows

In [ ]:
INSERT INTO [table_name] (
    [column1_name],
    [column2_name],
    [column3_name]
) VALUES (
    [value1],
    [value2],
    [value3]
);

## UPDATE statement
The WHERE clause is optional, and can contain any expression that would be valid in a SELECT statement.

In [ ]:
UPDATE [table_name]
SET [column_name] = [expression]
WHERE [expression]

## SET 
There are several variations we can use for our SET clause. First we can use a single value:

In [ ]:
UPDATE customer
SET phone = "+55 (12) 3921-4464"
WHERE customer_id = 1

# We can use a subquery that returns a single value:
UPDATE track
SET unit_price = (
                    SELECT AVG(unit_price)
                    FROM track
                 )

# We can use a column, or function on an existing column:
UPDATE track
SET unit_price = unit_price * 1.1

# Lastly, we can set more than one column at once:
UPDATE wishlist_track
SET
    active = 1,
    purchased = 0;

## DELETE rows

In [ ]:
DELETE FROM [table_name]

## Limit results

In [ ]:
SELECT * 
FROM recent_grads
LIMIT 5

## GROUP BY

In [ ]:
## 2. Calculating Group-Level Summary Statistics ##

SELECT major_category, AVG(ShareWomen)
FROM recent_grads
GROUP BY Major_category

SELECT major_category, AVG(Employed)/AVG(Total) as "share_employed"
FROM recent_grads
GROUP BY Major_category

## HAVING
Querying Virtual Columns With the HAVING Statement

In [ ]:
SELECT Major_category, AVG(Low_wage_jobs)/AVG(Total) as share_low_wage
FROM recent_grads
GROUP BY Major_category
HAVING share_low_wage > 0.1

## ROUND() Function ##

In [ ]:
SELECT ROUND(ShareWomen, 4), Major_category
FROM recent_grads

SELECT Major_category, ROUND(AVG(College_jobs)/AVG(Total),3) as share_degree_jobs
FROM recent_grads
GROUP BY Major_category
HAVING share_degree_jobs < 0.3

## Casting ##

In [ ]:
SELECT Major_category, SUM(CAST(Women as Float))/SUM(CAST(Total as Float)) SW
FROM recent_grads
GROUP BY Major_category
ORDER BY SW

## Aggregate Functions

In [ ]:
## 2. Finding a Columns Minimum and Maximum Values in SQL ##

SELECT Major, Major_category, MIN(Median)
FROM recent_grads
Where major_category="Engineering"

## 3. Calculating Sums and Averages in SQL ##

SELECT SUM(Total)
FROM recent_grads

## 4. Combining Multiple Aggregation Functions ##

SELECT AVG(Total), MIN(Men), MAX(Women)
from recent_grads

## 5. Customizing The Results ##

SELECT COUNT(*) as "Number of Students", MAX(Unemployment_rate) as "Highest Unemployment Rate"
FROM recent_grads

## 7. Performing Arithmetic in SQL ##

SELECT Major, Major_category, P75th - P25th "quartile_spread"
FROM recent_grads
ORDER BY quartile_spread
LIMIT 20

## Counting Unique Values ##

In [ ]:
SELECT COUNT(DISTINCT Major) "unique_majors", COUNT(DISTINCT Major_category) "unique_major_categories", COUNT(DISTINCT Major_Code) unique_major_codes
from recent_grads

## Left Join
A left join includes all the rows that an inner join will select, plus any joins from the first (or left) table that don't have a match in the second table. 

In [ ]:
SELECT f.name country, f.population
FROM facts f
LEFT JOIN cities c ON c.facts_id = f.id WHERE c.id IS NULL

## Inner Joins

An inner join works by including only rows from each table that have a match as specified using the ON clause. 
We don't need to use either column from our ON clause in our final list of columns. 
It's important whenever you use inner joins to be mindful that you might be excluding important data, especially if you are joining based on columns that aren't linked in the database schema.


In [ ]:
## 2. Understanding Inner Joins ##

SELECT c.*, f.name country_name FROM facts f
INNER JOIN cities c ON c.facts_id = f.id
LIMIT 5

## 3. Practicing Inner Joins ##

SELECT f.name country, c.name capital_city
FROM facts f
INNER JOIN cities c ON c.facts_id = f.id WHERE c.capital=1

## 2. Joining Three Tables ##

SELECt 
    t.track_id, 
    t.name track_name, 
    mt.name track_type, 
    il.unit_price, 
    il.quantity 
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN media_type mt ON t.media_type_id = mt.media_type_id
WHERE il.invoice_id = 4

## 3. Joining More Than Three Tables ##

SELECT
    il.track_id,
    t.name track_name,
    at.name artist_name,
    mt.name track_type,
    il.unit_price,
    il.quantity
FROM invoice_line il
INNER JOIN track t ON t.track_id = il.track_id
INNER JOIN media_type mt ON mt.media_type_id = t.media_type_id
INNER JOIN album a ON a.album_id = t.album_id
INNER JOIN artist at ON at.artist_id = a.artist_id
WHERE il.invoice_id = 4;

## UNION
Where regular joins are used to join columns, the union operator is used to join rows from tables and/or views.
the statements before and after UNION must have the same number of columns, with compatible types in order.

In [ ]:
SELECT * FROM chinook.customer_usa
UNION
SELECT * FROM chinook.customer_gt_90_dollars

## Joins with Subqueries

In [ ]:
SELECT c.name capital_city, f.name country, c.population
FROM facts f
INNER JOIN (SELECT * FROM cities 
            WHERE population >10000000 AND capital=1
            ) c ON c.facts_id = f.id
ORDER BY c.population DESC



## 8. Challenge: Complex Query with Joins and Subqueries ##

SELECT f.name country, c.urban_pop, f.population total_pop, CAST(urban_pop as float)/CAST(f.population as float) urban_pct
FROM facts f 
INNER JOIN (SELECT facts_id, SUM(population) urban_pop 
            from cities
           GROUP BY facts_id) 
            c ON c.facts_id = f.id
WHERE urban_pct > 0.5
ORDER BY urban_pct


## 4. Combining Multiple Joins with Subqueries ##

SELECT
    ta.album_name album,
    ta.artist_name artist,
    COUNT(*) tracks_purchased
FROM invoice_line il
INNER JOIN (
            SELECT
                t.track_id,
                ar.name artist_name,
                al.title album_name
            FROM track t
            INNER JOIN album al ON al.album_id = t.album_id
            INNER JOIN artist ar ON ar.artist_id = al.artist_id
           ) ta
           ON ta.track_id = il.track_id
GROUP BY 1
ORDER BY 3 DESC LIMIT 5;

## 7. Multiple Named Subqueries ##

WITH
    india_customers AS
        (
            SELECT *
            FROM chinook.customer
            WHERE country = "India"
         ),
    sum_customers AS
        (
            SELECT sum(total) total_purchases,
                customer_id
            FROM chinook.invoice
            GROUP BY customer_id
            )
SELECT 
    ic.first_name || " " || ic.last_name customer_name,
    sc.total_purchases
FROM india_customers AS ic
INNER JOIN (SELECT * FROM sum_customers) 
            AS sc ON ic.customer_id = sc.customer_id
ORDER BY 1

## the WITH clause

In [ ]:
WITH track_playlist AS
        (
        SELECT
            p.playlist_id,
            p.name AS playlist_name,
            t.name AS track_name,
            t.milliseconds/1000 AS length_seconds
        FROM playlist p
        LEFT JOIN playlist_track pt ON p.playlist_id = pt.playlist_id
        LEFT JOIN track t ON t.track_id = pt.track_id
        )

SELECT 
    playlist_id,
    playlist_name,
    COUNT(track_name) AS number_of_tracks,
    SUM(length_seconds) AS length_seconds
FROM track_playlist
GROUP BY playlist_id, playlist_name
ORDER BY playlist_id

## Combining Rows Using Intersect and Except ##

In [ ]:
WITH customers_usa_gt_90 AS
    (
    SELECT * FROM chinook.customer_usa
    INTERSECT
    SELECT * FROM chinook.customer_gt_90_dollars
    ) 
            
SELECT  e.first_name || " " || e.last_name AS employee_name,
        count(c.customer_id) AS customers_usa_gt_90
FROM chinook.employee e
LEFT JOIN customers_usa_gt_90 AS c ON e.employee_id = c.support_rep_id
WHERE e.title = "Sales Support Agent"
GROUP BY 1
ORDER BY 1

## Recursive Joins ##

In [ ]:
SELECT e1.first_name|| " " || e1.last_name employee_name, 
    e1.title employee_title, 
    e2.first_name || " " || e2.last_name supervisor_name, 
    e2.title supervisor_title
FROM employee e1
LEFT JOIN employee e2 ON e1.reports_to = e2.employee_id
ORDER BY employee_name

## Creating Views

In [ ]:
CREATE VIEW chinook.customer_gt_90_dollars AS
    SELECT c.* 
    FROM chinook.invoice i 
    INNER JOIN chinook.customer c ON i.customer_id = c.customer_id
    GROUP BY 1
    HAVING SUM(i.total) > 90;

SELECT * FROM chinook.customer_gt_90_dollars;


## Case Statements
Used here to generate columns

In [ ]:


SELECT c.first_name || " " || c.last_name customer_name,
    count(i.invoice_id) number_of_purchases,
    sum(i.total) total_spent,
    CASE
        WHEN sum(i.total) < 40 THEN "small spender"
        WHEN sum(i.total) > 100 THEN "big spender"
        ELSE "regular"
        END
        AS customer_category
FROM customer c
INNER JOIN invoice i ON i.customer_id = c.customer_id
GROUP BY 1
ORDER BY 1